In [ ]:
## imports
import os
os.chdir(os.pardir)

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd
from scipy.io import loadmat

from pycbem.field import efield, hfield
from pycbem.utils.dataloader import (load_antenna_el_properties,
                                     load_sphere_coords)
from pycbem.utils.viz import fig_config

In [ ]:
jax.config.update("jax_enable_x64", True)

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
# frequency
f = 10e9

# effective radiated skin area
target_area = (0.02, 0.02)  # 2 x 2 cm2
A = target_area[0] * target_area[1]
d = - 15 / 1000  # distance from the antenna
target_area_origin = (-target_area[0]/2, -target_area[1]/2)

# human head -- homogenous sphere model
r_c = load_sphere_coords(1304)

# antenna electric properties (Poljak 2005)
antenna_data = load_antenna_el_properties(f)
Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j
xs = antenna_data.x.to_numpy()
xs = xs - xs.max() / 2
ys = np.zeros_like(xs) + r_c['y'].min() + d
zs = np.zeros_like(xs)

In [ ]:
# E field
E = r_c.apply(
    lambda row: efield(row['x'], row['y'], row['z'], xs, ys, zs, Is, f),
    axis=1, result_type='expand')
E.columns = ['Ex', 'Ey', 'Ez']
E_abs = E.apply(
    lambda row: np.sqrt(row['Ex'] ** 2 + row['Ey'] ** 2 + row['Ez'] ** 2),
    axis=1)
E.loc[:, 'E_abs'] = E_abs

# H field
H = r_c.apply(
    lambda row: hfield(row['x'], row['y'], row['z'], xs, ys, zs, Is, f),
    axis=1, result_type='expand')
H.columns = ['Hx', 'Hy', 'Hz']
H_abs = H.apply(
    lambda row: np.sqrt(row['Hx'] ** 2 + row['Hy'] ** 2 + row['Hz'] ** 2),
    axis=1)
H.loc[:, 'H_abs'] = H_abs

# update dataframe
r_c_calc = pd.concat([r_c, E, H], axis=1)

# power density
def _pd(row):
    return (row['Ey'] * row['Hz'].conjugate()
            - row['Ez'] * row['Hy'].conjugate(),
            row['Ex'] * row['Hz'].conjugate(),
            row['Ex'] * row['Hy'].conjugate())

S = r_c_calc.apply(_pd, axis=1, result_type='expand')
S.columns = ['Sx', 'Sy', 'Sz']
S_abs = S.apply(
    lambda row: np.sqrt(row['Sx'] ** 2 + row['Sy'] ** 2 + row['Sz'] ** 2),
    axis=1)
S.loc[:, 'S_abs'] = S_abs 

# update dataframe
r_c_calc = pd.concat([r_c_calc, S], axis=1)

# effective area
r_c_calc_ta = r_c_calc[
    (r_c_calc['y'] < 0) &
    (r_c_calc['x'] > target_area_origin[0]) &
    (r_c_calc['x'] < target_area_origin[0] * -1) &
    (r_c_calc['z'] > target_area_origin[1]) &
    (r_c_calc['z'] < target_area_origin[1] * -1)]
r_c_calc_ta.reset_index(drop=True, inplace=True)

In [ ]:
fig_config(latex=True, scaler=2.5)
fig = plt.figure()
ax = plt.axes(projection ='3d')

cs = ax.scatter(r_c_calc['x'], r_c_calc['y'], r_c_calc['z'],
                c=np.abs(r_c_calc['S_abs']))
cbar = fig.colorbar(cs, ax=ax)
cbar.ax.set_ylabel('$S$ [W/m$^2$]')

ax.plot(xs, ys, zs, linestyle='-', c='r', zorder=3,
        label=(f'$\\lambda/2$ dipole antenna\n'
               f'$d = {-d * 1000}$ mm\n'
               f'$f = {f / 1e9}$ GHz'))

ax.set(xlabel='$x$', ylabel='$y$', zlabel='z')
fig.legend(bbox_to_anchor=(0.5, 0.9), ncol=1, frameon=True, edgecolor='k')
plt.show()

In [ ]:
fig_config(latex=True, scaler=2)
fig = plt.figure()
ax = fig.add_subplot()
cs = ax.scatter(r_c_calc['x'], r_c_calc['z'], c=np.abs(r_c_calc['S_abs']))
cbar = fig.colorbar(cs, ax=ax)
cbar.ax.set_ylabel('$S$ [W/m$^2$]')
ax.plot(xs, zs, linestyle='-', c='r',
        label=f'$\\lambda/2$ dipole antenna')
ax.add_patch(Rectangle(target_area_origin, target_area[0], target_area[1],
                       ec='w', fc='None',
                       label=f'effective area\n$A = {A * 1e4}$ cm$^2$'))
ax.set(xlabel='$x$', ylabel='z')
ax.legend(facecolor='darkslateblue', labelcolor='white', framealpha=0.9)
plt.show()